In [ ]:
import pymongo
from sklearn.ensemble import IsolationForest

# MongoDB connection details
MONGO_URI = "mongodb://localhost:27017/"
COLLECTION_NAME = "slv_notasfiscaisgit"



In [ ]:
# Connect to MongoDB
client = pymongo.MongoClient(MONGO_URI)
db = client["<database>"]
collection = db[COLLECTION_NAME]

# Extract relevant features for anomaly detection
data = []
for document in collection.find():
    features = [
        int(document["CNPJ EMITENTE"]),
        int(document["CNPJ DESTINATÁRIO"]),
        int(document["VALOR NOTA FISCAL"]),
    ]
    data.append(features)

# Train the Isolation Forest model
model = IsolationForest(contamination="auto")  # Adjust contamination as needed
model.fit(data)

# Predict anomaly scores
anomaly_scores = model.decision_function(data)

# Identify anomalies based on a threshold
threshold = 0  # Adjust threshold as needed
anomalies = [i for i, score in enumerate(anomaly_scores) if score < threshold]

# Print the anomalous documents
print("Anomalous documents:")
for index in anomalies:
    document = collection.find().skip(index).limit(1)[0]
    print(document)

# Close the MongoDB connection
client.close()